Procesamiento Refined

In [12]:
#Importamos Pandas y cargamos archivos
import pandas as pd

# Cargamos el archivo de medallas
df_medals = pd.read_csv('./datos/raw/olympic_medals.csv')

# Cargamos el archivo de resultados
df_results = pd.read_csv('./datos/raw/olympic_results_from_pkl.csv')

Definimos Funciones a Utilizar

In [13]:
#funcion para sutituir un valor nulo de una columna por un valor existente de la misma columna buscando por otro codigo relacionado
def sustituir_valor_filtrado(df, value, filter):
    # Identificar las filas que tienen valor nulo en la columna 'value'
    mask = pd.isnull(df[value])    
    # Filtrar el DataFrame original para obtener solo las filas donde 'value' es nulo
    df_nulls = df[mask]  
    # Obtener los valores únicos de 'filter' para iterar sobre ellos
    unique_filters = df_nulls[filter].unique() 
    # Iterar sobre cada valor único de 'filter' y rellenar los valores nulos correspondientes
    for f in unique_filters:
        # Filtrar las filas donde 'filter' coincide y 'value' no es nulo
        mask_filter = (df[filter] == f) & pd.notna(df[value])
        df_filter = df[mask_filter]
        
        # Si hay filas en df_filter, asignar el valor de la primera fila a las filas correspondientes en df_nulls
        if not df_filter.empty:
            first_value = df_filter.iloc[0][value]
            df.loc[mask & (df[filter] == f), value] = first_value
    
    return df

Limpieza Dataset Medallas

In [14]:
#Remplazamos columna country_cod nulas por columna country_cod no nulas cuando country_3_letter_code es igual en ambas filas
df_medals_cc = sustituir_valor_filtrado(df_medals, 'country_code', 'country_3_letter_code')

#Si aun existen datos en la columna country_code nulos les asignamos el valor de la columna country_3_letter_code
df_medals_cc['country_code'].fillna(df_medals_cc['country_3_letter_code'], inplace = True)

# Ya que en muchas de las filas la columna participant_title es igual a la columna country_name, sustituimos participant_title nulos por country_name

#Continuamos a partir del dataframe con la columna country_code sin valores nulos
df_medals_pt = df_medals_cc
df_medals_pt['participant_title'].fillna(df_medals_pt['country_name'], inplace=True)


#Ya que la url es un valor fijo + el nombre del atleta, cuando la url del atleta es nula y el nombre no lo es, parseamos el nombre para darle un formato especifico y lo concatenamos al valor fijo de la url

#Continuamos a partir del dataframe con la columna country_code y participant_title sin valores nulos
df_medals_au = df_medals_pt

fix = 'https://olympics.com/en/athletes/'

#Recorremos el dataframe
for i in range(len(df_medals_au)):
    fila = df_medals_au.iloc[i]
    
    if((pd.notna(fila['athlete_full_name'])) & pd.isnull(fila['athlete_url'])):
    #Si la url es nula y el nombre del atleta no, parseamos el nombre a minusculas, sustituimos los espacios por un guion y concatenamos el valor fijo de la url + el nombre
        nom = fila['athlete_full_name'].lower().replace(" ", "-")
        url = fix+nom
        df_medals_au.at[i, 'athlete_url'] = url

#Por ultimo rellenamos los valores nulos faltantes con el dato vacio ''
df_clean_medals = df_medals_au.fillna('')

display(df_clean_medals.isnull().sum())

discipline_title         0
slug_game                0
event_title              0
event_gender             0
medal_type               0
participant_type         0
participant_title        0
athlete_url              0
athlete_full_name        0
country_name             0
country_code             0
country_3_letter_code    0
dtype: int64

Limpieza Dataset Resultados

In [16]:
#Completamos los las filas sin medallas con 'NA'
df_results_mt = df_results
df_results_mt['medal_type'].fillna('NA', inplace = True)

#Completamos los las filas sin atletas con ''
df_results_a = df_results_mt
df_results_a['athletes'].fillna('', inplace = True)

#Ya que la url es un valor fijo + el nombre del atleta, cuando es un atleta, la url es nula y el nombre no lo es, 
#parseamos el nombre para darle un formato especifico y lo concatenamos al valor fijo de la url
df_results_url = df_results_a

fix = 'https://olympics.com/en/athletes/'

for i in range(len(df_results_url)):
    fila = df_results_url.iloc[i]
    
    if((fila['participant_type'] == 'Athlete') & (pd.notna(fila['athlete_full_name'])) & pd.isnull(fila['athlete_url'])):
        nom = fila['athlete_full_name'].lower().replace(" ", "-")
        url = fix+nom
        df_results_url.at[i, 'athlete_url'] = url

#Si es un equipo y el atleta y la url son nulas sustituimos por ''
df_results_ua = df_results_a

condicion = df_results_ua['participant_type'] == 'GameTeam'
df_results_ua.loc[condicion, ['athlete_url', 'athlete_full_name']] = df_results_ua.loc[condicion, ['athlete_url', 'athlete_full_name']].fillna('')


#Sustituimos rank_equal por False cuando es nulo 
df_results_re = df_results_ua
df_results_re['rank_equal'].fillna(False, inplace = True)

#Sustituimos rank_position por NM(No Mark) cuando es nulo 
df_results_rp = df_results_re
df_results_rp['rank_position'].fillna('NM', inplace = True)

#Remplazamos columna country_cod nulas por columna country_cod no nulas cuando country_3_letter_code es igual en ambas filas
df_results_cc = sustituir_valor_filtrado(df_results_rp, 'country_code', 'country_3_letter_code')

#Si aun existen datos en la columna country_code nulos les asignamos el valor de la columna country_3_letter_code
df_results_cc['country_code'].fillna(df_results_cc['country_3_letter_code'], inplace = True)

#Remplazamos columna value_type nulas por columna value_type no nulas cuando discipline_title es igual en ambas filas
df_results_vt = sustituir_valor_filtrado(df_results_cc, 'value_type', 'discipline_title')

#Si aun existen datos en la columna value_type nulos les asignamos el valor NT(No Type)
df_results_vt['value_type'].fillna('NT', inplace = True)

#Sustituimos value_unit por NV(No Value) cuando es nulo 
df_results_vu = df_results_vt
df_results_vu['value_unit'].fillna('NV', inplace = True)

#Por ultimo rellenamos los valores nulos faltantes con el dato vacio ''
df_clean_results = df_results_vu.fillna('')

display(df_clean_results.isnull().sum())

Unnamed: 0               0
discipline_title         0
event_title              0
slug_game                0
participant_type         0
medal_type               0
athletes                 0
rank_equal               0
rank_position            0
country_name             0
country_code             0
country_3_letter_code    0
athlete_url              0
athlete_full_name        0
value_unit               0
value_type               0
dtype: int64